In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
mimic_iv_path = "/cis/home/charr165/Documents/physionet.org/mimiciv/2.2"
mm_dir = "/cis/home/charr165/Documents/multimodal"

output_dir = os.path.join(mm_dir, "preprocessing")
os.makedirs(output_dir, exist_ok=True)

In [4]:
mm_dir = "/cis/home/charr165/Documents/multimodal"
output_dir = os.path.join(mm_dir, "preprocessing")

# vitals_ts_df = pd.read_pickle(os.path.join(output_dir, "ts_vitals_icu.pkl"))
# procedureevents_ts_df = pd.read_pickle(os.path.join(output_dir, "ts_procedureevents_icu.pkl"))
# ts_labs_icu = pd.read_pickle(os.path.join(output_dir, "ts_labs_icu.pkl"))

# labs_vitals_ts_df = pd.read_pickle(os.path.join(output_dir, "ts_labs_vitals_icu.pkl"))
labs_vitals_ts_df = pd.read_pickle(os.path.join(output_dir, "ts_labs_vitals.pkl"))
labs_vitals_ts_df = labs_vitals_ts_df[labs_vitals_ts_df['icu_time_delta'] > 0]
labs_vitals_ts_df.drop(columns=['hosp_time_delta'], inplace=True)
labs_vitals_ts_df.rename(columns={'icu_time_delta': 'timedelta'}, inplace=True)

In [5]:
def impute_values_optimized(df):
    interval_length = 1
    global_means = df.mean()
    grouped = df.groupby('stay_id')

    all_stays_imputed = []

    for stay_id, group in tqdm(grouped):
        group = group.copy()
        
        group['time_interval'] = (group['timedelta'] / interval_length).astype(int)

        curr_stay_imputed = group.groupby('time_interval').last()

        curr_stay_imputed = curr_stay_imputed.reindex(range(curr_stay_imputed.index.max() + 1)).ffill()
        curr_stay_imputed['timedelta'] = curr_stay_imputed.index * interval_length
        curr_stay_imputed.fillna(global_means, inplace=True)

        all_stays_imputed.append(curr_stay_imputed)

    imputed_df = pd.concat(all_stays_imputed, axis=0, ignore_index=True)

    return imputed_df

# imputed_vitals = impute_values_optimized(vitals_ts_df)
imputed_labs_vitals = impute_values_optimized(labs_vitals_ts_df)


100%|██████████| 73171/73171 [07:17<00:00, 167.20it/s]


In [6]:
# imputed_labs_vitals.to_pickle(os.path.join(output_dir, "imputed_ts_labs_vitals_icu.pkl"))
imputed_labs_vitals.to_pickle(os.path.join(output_dir, "imputed_ts_labs_vitals.pkl"))
# imputed_vitals.to_pickle(os.path.join(output_dir, "imputed_ts_vitals_icu.pkl"))